把原始資料和氣象站資合併，並包成新的csv

In [202]:
import pandas as pd
from datetime import datetime, timedelta

data_dir = 'OriginalData'
df = pd.read_csv(f'..//Data//{data_dir}//L1_Train.csv')
df

,LocationCode,DateTime,WindSpeed(m/s),Pressure(hpa),Temperature(°C),Humidity(%),Sunlight(Lux),Power(mW)
0,1,2024-01-01 06:31:08.000,0.0,1016.50,17.5,86.8,25.00,0.0
1,1,2024-01-01 06:32:08.000,0.0,1016.53,17.5,86.7,28.33,0.0
2,1,2024-01-01 06:33:08.000,0.0,1016.57,17.5,86.7,32.50,0.0
3,1,2024-01-01 06:34:08.000,0.0,1016.58,17.5,86.7,39.17,0.0
4,1,2024-01-01 06:35:08.000,0.0,1016.59,17.5,86.7,45.83,0.0
...,...,...,...,...,...,...,...,...
101668,1,2024-07-24 18:17:07.000,0.0,959.80,25.1,100.0,23.33,0.0
101669,1,2024-07-24 18:18:07.000,0.0,959.92,25.1,100.0,23.33,0.0
101670,1,2024-07-24 18:19:07.000,0.0,960.09,25.1,100.0,33.33,0.0
101671,1,2024-07-24 18:20:07.000,0.0,959.40,25.1,100.0,40.00,0.0


In [203]:
df.columns.to_list()

['LocationCode',
 'DateTime',
 'WindSpeed(m/s)',
 'Pressure(hpa)',
 'Temperature(°C)',
 'Humidity(%)',
 'Sunlight(Lux)',
 'Power(mW)']

In [204]:
df['DateTime'] = pd.to_datetime(df['DateTime'])
df['Year'] = df['DateTime'].dt.year
df['Month'] = df['DateTime'].dt.month
df['Day'] = df['DateTime'].dt.day
df['Hour'] = df['DateTime'].dt.hour
df['Time'] = df['DateTime'].dt.strftime('%M:%S')

period = {
    '00': ("00:00", "09:59"),
    '10': ("10:00", "19:59"),
    '20': ("20:00", "29:59"),
    '30': ("30:00", "39:59"),
    '40': ("40:00", "49:59"),
    '50': ("50:00", "59:59")
}

periods = {
    key: (
        datetime.strptime(value[0], "%M:%S").time(),
        datetime.strptime(value[1], "%M:%S").time()
    )
    for key, value in period.items()
}

def get_period(time_str):
    time = datetime.strptime(time_str, "%M:%S").time()
    
    for period, (start, end) in periods.items():
        if start <= time <= end:
            return period
    return None

df['Minute'] = df['Time'].apply(get_period)
df

,LocationCode,DateTime,WindSpeed(m/s),Pressure(hpa),Temperature(°C),Humidity(%),Sunlight(Lux),Power(mW),Year,Month,Day,Hour,Time,Minute
0,1,2024-01-01 06:31:08,0.0,1016.50,17.5,86.8,25.00,0.0,2024,1,1,6,31:08,30
1,1,2024-01-01 06:32:08,0.0,1016.53,17.5,86.7,28.33,0.0,2024,1,1,6,32:08,30
2,1,2024-01-01 06:33:08,0.0,1016.57,17.5,86.7,32.50,0.0,2024,1,1,6,33:08,30
3,1,2024-01-01 06:34:08,0.0,1016.58,17.5,86.7,39.17,0.0,2024,1,1,6,34:08,30
4,1,2024-01-01 06:35:08,0.0,1016.59,17.5,86.7,45.83,0.0,2024,1,1,6,35:08,30
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
101668,1,2024-07-24 18:17:07,0.0,959.80,25.1,100.0,23.33,0.0,2024,7,24,18,17:07,10
101669,1,2024-07-24 18:18:07,0.0,959.92,25.1,100.0,23.33,0.0,2024,7,24,18,18:07,10
101670,1,2024-07-24 18:19:07,0.0,960.09,25.1,100.0,33.33,0.0,2024,7,24,18,19:07,10
101671,1,2024-07-24 18:20:07,0.0,959.40,25.1,100.0,40.00,0.0,2024,7,24,18,20:07,20


In [205]:
to_average_features = [
    'WindSpeed(m/s)',
    'Pressure(hpa)',
    'Temperature(°C)',
    'Humidity(%)',
    'Sunlight(Lux)',
    'Power(mW)',
    ]
avg_period = df.groupby(['Year', 'Month', 'Day', 'Hour', 'Minute'])[to_average_features].mean().round(2).reset_index()
avg_df = pd.DataFrame(avg_period)
avg_df

,Year,Month,Day,Hour,Minute,WindSpeed(m/s),Pressure(hpa),Temperature(°C),Humidity(%),Sunlight(Lux),Power(mW)
0,2024,1,1,6,30,0.0,1016.54,17.50,86.78,50.09,0.00
1,2024,1,1,6,40,0.0,1016.59,17.49,86.80,163.25,0.02
2,2024,1,1,6,50,0.0,1016.66,17.50,86.95,326.83,0.05
3,2024,1,1,7,00,0.0,1016.73,17.50,86.99,470.83,0.08
4,2024,1,1,7,10,0.0,1016.77,17.57,86.97,659.67,0.14
...,...,...,...,...,...,...,...,...,...,...,...
10620,2024,7,24,17,40,0.0,961.30,25.22,100.00,91.25,0.00
10621,2024,7,24,17,50,0.0,961.06,24.91,100.00,54.00,0.00
10622,2024,7,24,18,00,0.0,960.91,24.76,100.00,37.25,0.00
10623,2024,7,24,18,10,0.0,959.86,24.97,100.00,40.33,0.00


In [206]:
save_dir = 'PreparedData'
avg_df.to_csv(f"..//Data//{save_dir}//L1_Processed_Train.csv",index=False)

In [207]:
"""轉SeqNumber"""
Year = avg_df['Year'].astype(str)
Month = avg_df['Month'].astype(str).str.zfill(2)
Day = avg_df['Day'].astype(str).str.zfill(2)
Hour = avg_df['Hour'].astype(str).str.zfill(2)
Minute = avg_df['Minute'].astype(str).str.zfill(2)

DeviceID = df['LocationCode'].astype(str).str.zfill(2)

drop_features=[
    'Year',
    'Month',
    'Day',
    'Hour',
    'Minute'
]
avg_df.insert(
    0, 
    'SeqNumber', 
    Year + Month + Day + Hour + Minute + DeviceID
    )
avg_df.drop(columns=drop_features,inplace=True)
avg_df

,SeqNumber,WindSpeed(m/s),Pressure(hpa),Temperature(°C),Humidity(%),Sunlight(Lux),Power(mW)
0,20240101063001,0.0,1016.54,17.50,86.78,50.09,0.00
1,20240101064001,0.0,1016.59,17.49,86.80,163.25,0.02
2,20240101065001,0.0,1016.66,17.50,86.95,326.83,0.05
3,20240101070001,0.0,1016.73,17.50,86.99,470.83,0.08
4,20240101071001,0.0,1016.77,17.57,86.97,659.67,0.14
...,...,...,...,...,...,...,...
10620,20240724174001,0.0,961.30,25.22,100.00,91.25,0.00
10621,20240724175001,0.0,961.06,24.91,100.00,54.00,0.00
10622,20240724180001,0.0,960.91,24.76,100.00,37.25,0.00
10623,20240724181001,0.0,959.86,24.97,100.00,40.33,0.00
